In [3]:
import tensorflow as tf
import numpy as np

**`tf.clip_by_average_norm`**
**`tf.clip_by_global_norm`**
**`tf.clip_by_norm`**
**`tf.clip_by_value(t, clip_value_min, clip_value_max, name=None)`**
+ Clips tensor values to a specified min and max.

In [8]:
v = tf.constant([[1.0, 2.0, 4.0],[4.0, 5.0, 6.0]])
result = tf.clip_by_value(v, 2.5, 4.5)

with tf.Session() as sess:
    print(sess.run(v))
    print(sess.run(result))

[[1. 2. 4.]
 [4. 5. 6.]]
[[2.5 2.5 4. ]
 [4.  4.5 4.5]]


In [ ]:
tf.train

In [ ]:
tf.train.Optimizer()
The Optimizer base class provides methods to compute gradients for a loss and apply gradients to variables.

Base class for optimizers.

This class defines the API to add Ops to train a model.  You never use this
class directly, but instead instantiate one of its subclasses such as
`GradientDescentOptimizer`, `AdagradOptimizer`, or `MomentumOptimizer`.

### Usage

```python
# Create an optimizer with the desired parameters.
opt = GradientDescentOptimizer(learning_rate=0.1)
# Add Ops to the graph to minimize a cost by updating a list of variables.
# "cost" is a Tensor, and the list of variables contains tf.Variable
# objects.
opt_op = opt.minimize(cost, var_list=<list of variables>)
```

In the training program you will just have to run the returned Op.

```python
# Execute opt_op to do one step of training:
opt_op.run()
```

### Processing gradients before applying them.

Calling `minimize()` takes care of both computing the gradients and
applying them to the variables.  If you want to process the gradients
before applying them you can instead use the optimizer in three steps:

1.  Compute the gradients with `compute_gradients()`.
2.  Process the gradients as you wish.
3.  Apply the processed gradients with `apply_gradients()`.

Example:

```python
# Create an optimizer.
opt = GradientDescentOptimizer(learning_rate=0.1)

# Compute the gradients for a list of variables.
grads_and_vars = opt.compute_gradients(loss, <list of variables>)

# grads_and_vars is a list of tuples (gradient, variable).  Do whatever you
# need to the 'gradient' part, for example cap them, etc.
capped_grads_and_vars = [(MyCapper(gv[0]), gv[1]) for gv in grads_and_vars]

# Ask the optimizer to apply the capped gradients.
opt.apply_gradients(capped_grads_and_vars)
```

### Gating Gradients

Both `minimize()` and `compute_gradients()` accept a `gate_gradients`
argument that controls the degree of parallelism during the application of
the gradients.

The possible values are: `GATE_NONE`, `GATE_OP`, and `GATE_GRAPH`.

<b>`GATE_NONE`</b>: Compute and apply gradients in parallel.  This provides
the maximum parallelism in execution, at the cost of some non-reproducibility
in the results.  For example the two gradients of `matmul` depend on the input
values: With `GATE_NONE` one of the gradients could be applied to one of the
inputs _before_ the other gradient is computed resulting in non-reproducible
results.

<b>`GATE_OP`</b>: For each Op, make sure all gradients are computed before
they are used.  This prevents race conditions for Ops that generate gradients
for multiple inputs where the gradients depend on the inputs.

<b>`GATE_GRAPH`</b>: Make sure all gradients for all variables are computed
before any one of them is used.  This provides the least parallelism but can
be useful if you want to process all gradients before applying any of them.

### Slots

Some optimizer subclasses, such as `MomentumOptimizer` and `AdagradOptimizer`
allocate and manage additional variables associated with the variables to
train.  These are called <i>Slots</i>.  Slots have names and you can ask the
optimizer for the names of the slots that it uses.  Once you have a slot name
you can ask the optimizer for the variable it created to hold the slot value.

This can be useful if you want to log debug a training algorithm, report stats
about the slots, etc.
Init docstring:
Create a new Optimizer.

This must be called by the constructors of subclasses.

Args:
  use_locking: Bool. If True apply use locks to prevent concurrent updates
    to variables.
  name: A non-empty string.  The name to use for accumulators created
    for the optimizer.

Raises:
  ValueError: If name is malformed.

**`tf.train.AdamOptimizer`**
**`tf.train.ExponentialMovingAverage`**
**`tf.train.GradientDescentOptimizer`**
**`tf.train.MomentumOptimizer`**
**`tf.train.Optimizer`**
**`tf.train.RMSPropOptimizer`**
**`tf.train.XXX`**

In [ ]:
tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')

Construct a new Adam optimizer.

Initialization:

$$m_0 := 0  ext{(Initialize initial 1st moment vector)}$$
$$v_0 := 0  ext{(Initialize initial 2nd moment vector)}$$
$$t := 0    ext{(Initialize timestep)}$$

The update rule for `variable` with gradient `g` uses an optimization
described at the end of section2 of the paper:

$$t := t + 1$$
$$lr_t :=   ext{learning\_rate} * \sqrt{1 - beta_2^t} / (1 - beta_1^t)$$

$$m_t := beta_1 * m_{t-1} + (1 - beta_1) * g$$
$$v_t := beta_2 * v_{t-1} + (1 - beta_2) * g * g$$
$$variable := variable - lr_t * m_t / (\sqrt{v_t} + \epsilon)$$

The default value of 1e-8 for epsilon might not be a good default in
general. For example, when training an Inception network on ImageNet a
current good choice is 1.0 or 0.1. Note that since AdamOptimizer uses the
formulation just before Section 2.1 of the Kingma and Ba paper rather than
the formulation in Algorithm 1, the "epsilon" referred to here is "epsilon
hat" in the paper.

The sparse implementation of this algorithm (used when the gradient is an
IndexedSlices object, typically because of `tf.gather` or an embedding
lookup in the forward pass) does apply momentum to variable slices even if
they were not used in the forward pass (meaning they have a gradient equal
to zero). Momentum decay (beta1) is also applied to the entire momentum
accumulator. This means that the sparse behavior is equivalent to the dense
behavior (in contrast to some momentum implementations which ignore momentum
unless a variable slice was actually used).

Args:
  learning_rate: A Tensor or a floating point value.  The learning rate.
  beta1: A float value or a constant float tensor.
    The exponential decay rate for the 1st moment estimates.
  beta2: A float value or a constant float tensor.
    The exponential decay rate for the 2nd moment estimates.
  epsilon: A small constant for numerical stability. This epsilon is
    "epsilon hat" in the Kingma and Ba paper (in the formula just before
    Section 2.1), not the epsilon in Algorithm 1 of the paper.
  use_locking: If True use locks for update operations.
  name: Optional name for the operations created when applying gradients.
    Defaults to "Adam".

@compatibility(eager)
When eager execution is enabled, `learning_rate`, `beta1`, `beta2`, and
`epsilon` can each be a callable that takes no arguments and returns the
actual value to use. This can be useful for changing these values across
different invocations of optimizer functions.
@end_compatibility

In [ ]:
相比较于 SGD 算法而言
1. 不容易陷入局部最优点
2. 速度更快

learning_rate: A Tensor or a floating point value. The learning rate.
beta1: A float value or a constant float tensor. The exponential decay rate for the 1st moment estimates.
beta2: A float value or a constant float tensor. The exponential decay rate for the 2nd moment estimates.
epsilon: A small constant for numerical stability. This epsilon is "epsilon hat" in the Kingma and Ba paper (in the formula just before Section 2.1), not the epsilon in Algorithm 1 of the paper.
use_locking: If True use locks for update operations.
name: Optional name for the operations created when applying gradients. Defaults to "Adam".·

In [ ]:
**``**